# 0. 사전 세팅

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%cd "/content/drive/MyDrive/데이터 분석/projects/ML_portfolio/10_kleague_final_pass_prediction"

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 7. EDA 인사이트

---

▸ 타겟 구조

    에피소드별 마지막 이벤트는 항상 Pass이고, 그 Pass의 (end_x, end_y)가 타겟
    패스 성공/실패 비율 ≈ 56:44 정도로 아주 심하게 치우치지 않음
    거리 기준으로 보면 10~20m 구간의 패스 성공률이 가장 높고, 30m 이상부터 급격히 떨어짐

    < 인사이트 >
    - 순수 좌표 회귀(MAE/MSE) + 거리 관련 feature를 같이 쓰면 좋을 듯 !
    - multi-task로 distance_bin(0–10 / 10–20 / ...)이나 zone까지 같이 예측하게 하면 representation quality가 좋아질 수 있음

▸ 시퀀스/정렬 구조

    전체 에피소드 중 98% 이상은 시간 순서 정렬이 깔끔하고, 나머지 1.8% 정도만 역전이 있음
    대부분 -0.1 이내의 미세 오차, 진짜 심각한 역전은 적음

    < 인사이트 >
    - 시퀀스 모델은 써도 될 것 같고 !
    - 단, 전처리에서 time_seconds, action_id 기준으로 확실하게 정렬하고, 마스크 처리만 잘 해주면 됨
    - 몇 개 안 되는 진짜 이상치는 드롭하거나 별도 처리해도 전체 성능에 영향 거의 없음

▸ 이벤트 구성 패턴

    전체 이벤트 중 Pass ≈ 50%, Carry ≈ 23%, Turnover 관련(Recovery + Interception + Tackle + Duel) ≈ 15%
    최빈 bigram: Pass→Pass, Carry→Pass, Pass→Carry, Turnover→Pass
    최빈 trigram: Pass-Carry-Pass, Pass-Pass-Pass, Carry-Pass-Pass 등

    < 인사이트 >
    - n-gram 구조가 강해서 TCN(1D conv), RNN, Transformer 모두 잘 맞는 도메인
    - 특히 Carry → Pass, Turnover → Pass 같은 패턴은 final pass 위치와 전술 의도를 암시해줄 수 있음

▸ 마지막 이벤트 직전 패턴

    Final pass 바로 직전 이벤트의 90%가 Carry / Pass / Recovery 셋 중 하나
    평균 end_x/거리 기준으로 보면 Carry / Tackle 직후 패스가 가장 전방, 가장 먼 거리

    < 인사이트 >
    - prev_event_type, prev_event_dx/dy/angle는 무조건 써야하는 feature
    - 심지어 “마지막 3~4 스텝만 따로 뽑아서 쓰는 모델”도 하나의 strong baseline으로 가능할 듯

▸ Episode별 클러스터 (5개 패턴)

    Balanced build-up / 짧은 측면 전개 / 리셋/후퇴 패턴 / 짧은 반대 측면 전개 / 긴 빌드업 (dist_cum 가장 큼)
    마지막 패스 성공률까지 보면 Cluster 4 (Long Build-up)가 최상(~0.63)

    < 인사이트 >
    - episode에 붙는 cluster_id 자체가 embedding으로 사용될 수 있음
    - Mixture-of-experts / cluster-wise head 같은 구조도 고려 가능
    - 최소한 cluster_id를 one-hot 또는 embedding으로 넣으면, “지금의 빌드업 흐름이 어떤 종류인지” 모델이 한 번에 인식

▸ Player별 분석 결과

    선수별 carry_ratio 분포가 그렇게 극단적이지 않음
    선수별 angle_mean, final pass (end_x, end_y) mean 위치도 좁은 범위에 몰려 있고, 뚜렷한 클러스터 구조 없음

    < 인사이트 >
    - player_id embedding은 효과 대비 리스크(차원+노이즈) 가 큼
    - baseline에서는 아예 빼고 시작하는 게 합리적
    - 나중에 여유 있으면 작은 차원(8~16) + strong dropout으로 시험해보는 정도

▸ Episode별 움직임 & 각도 smoothness

    cum_dx, cum_dy로 episode가 전진 위주인지, 좌우 측면 전개인지가 드러남
    angle 변화량 기준으로 보면, 누적 전진량이 클수록 angle이 더 안정(episode가 한 방향으로 쭉 진행)

    < 인사이트 >
    - cum_dx, cum_dy, movement_norm, angle_mean_abs, angle_std는 전술 패턴을 대표하는 episode별 feature
    - final pass의 zone/거리/각도 예측에 직접적인 신호를 줌

▸ Turnover 이후 3-step window

    Turnover 직후 첫 행동은 dx ~ 0 (잡아두기), 그 이후 2~3 step에서 전진/측면 전개가 본격적으로 나타남

    < 인사이트 >
    - “turnover 이후 k-step” 여부를 표시하는 feature가 유용하게 쓰일 수 있음
    - 특히 final pass가 turnover 직후 짧은 시퀀스에서 나오는지, 긴 빌드업 끝에서 나오는지 구분해줄 수 있음

# 8. EDA에 이은 Feature Engineering (함수 정의)

---

▸ 패스 각도(angle)

    angle = arctan((end_y-start_y) / (end_x-start_x))

    풀백은 측면으로 많이 주고, 중앙 미드필더는 전진 패스의 비율 높음
    수비수는 옆으로 주는 패스나 후방 패스의 비중 높음

▸ 패스 진행 거리

    더 먼 패스일수록 progressive chance가 높고, end_x가 강하게 증가하는 패턴을 가짐

▸ event_type 임베딩

    type_name → embedding vector
    result_name → embedding vector

    sequence embedding에 필수적으로 진행해야하는 것

▸ 에피소드에서의 속도(Δx, Δy)

    dx_t = x_t - x_(t-1)
    dy_t = y_t - y_(t-1)

    엔드 투 엔드 모델보다 훨씬 패턴 학습이 잘 됨

    dx > 0 → 오른쪽으로 전진 중
    dy > 0 → 위쪽으로 이동 중
    dy < 0 → 아래쪽으로 이동 중
    dx ≈ 0 → 횡패스 빈도 높음
    dx < 0 → 후방 패스 비율 증가 (안정화)

    1. 한 에피소드에서 dx가 계속 증가한다 ➜ 공격 전개 중 (전진 패스 가능성이 높음)
    2. dy가 크게 증가했다➜ 측면 전개 중 (사이드로 패스가 날아갈 가능성)
    3. dx가 음수로 전환되었다 ➜ 후방 안정화 패스 패턴
    4. dx, dy가 급격히 바뀐다 ➜ 압박을 벗어나기 위한 빠른 전개

## 8.1 공통 Util & 기본 데이터 정렬

In [ ]:
import numpy as np
import pandas as pd

from collections import Counter
from math import log2

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

### 8.1.1 이벤트 단순화(클러스터링), Zone 함수 (EDA에서 쓰던 것 정리)

In [ ]:
# 이벤트 타입 단순화하는 함수
def simplify_event(t: str) -> str:
    # Pass 계열
    if t in ["Pass", "Pass_Freekick", "Pass_Corner"]:
        return "Pass"

    # Carry
    if t == "Carry":
        return "Carry"

    # Duel / Turnover 계열
    if t in ["Duel", "Tackle", "Interception", "Recovery"]:
        return "Duel_Turnover"

    # Cross
    if "Cross" in t:
        return "Cross"

    # Shot
    if "Shot" in t:
        return "Shot"

    # Clearance 계열
    if t in ["Clearance", "Aerial Clearance"]:
        return "Clearance"

    # GK Action 계열
    if t in ["Catch", "Parry", "Goal Kick", "Keeper Rush-Out"]:
        return "GK_Action"

    # Block / Deflection / Intervention (소유권 방해)
    if t in ["Block", "Deflection", "Intervention", "Hit"]:
        return "Deflect_Block"

    # Set-piece
    if t in ["Throw-In"]:
        return "SetPiece"

    # Goal 이벤트
    if t in ["Goal", "Own Goal", "Shot_Corner", "Shot_Freekick", "Penalty Kick"]:
        return "Goal_Event"

    # Error / Out / Foul 계열
    if t in ["Error", "Out", "Foul", "Foul_Throw", "Handball_Foul", "Offside"]:
        return "Error_Out"

    # 기타
    return "Misc"

# 이벤트 결과 단순화하는 함수
def simplify_result(result_name):
    if result_name in ["Successful", "On Target", "Goal"]:
        return "Success"

    if result_name in ["Unsuccessful", "Off Target", "Blocked"]:
        return "Fail"

    return "None"

# Zone 구분하는 함수
def get_zone_x(x):
    if x < 35: return "D3"
    elif x < 70: return "M3"
    else: return "A3"

def get_zone_y(y):
    if y < 22: return "Left"
    elif y < 45: return "Center"
    else: return "Right"

# 시퀀스(에피소드) 엔트로피 측정하는 함수
def sequence_entropy(seq):
    cnt = Counter(seq)
    total = len(seq)

    if total == 0:
        return 0.0

    probs = [c / total for c in cnt.values()]

    return -sum(p * log2(p) for p in probs if p > 0)

### 8.1.2 기본 정렬 함수

In [ ]:
SORT_COLS = ["game_id", "period_id", "episode_id", "time_seconds", "action_id"]

def sort_events(df: pd.DataFrame) -> pd.DataFrame:
    """
    time_seconds, action_id 기준으로 episode 내 이벤트 정렬.
    """
    df_sorted = df.sort_values(SORT_COLS).reset_index(drop=True)
    return df_sorted

## 8.2 이벤트별 Feature Engineering

---

    한 이벤트마다 어떤 Feature를 만들지를 담당하는 함수

### 8.2.1 Turnover flag 계산 (EDA에서 쓴 함수)

In [ ]:
def add_turnover_flag(df):
    df = df.copy()

    # Fail 정의
    fail = df["result_simple"] == "Fail"

    # Pass / Cross / SetPiece 실패 → turnover
    cond_fail_pass = df["event_simple"].isin(["Pass", "Cross", "SetPiece"]) & fail

    # Take-On 실패
    cond_takeon_fail = (df["type_name"] == "Take-On") & (df["result_name"] == "Unsuccessful")

    # Duel 실패
    cond_duel_fail = (df["type_name"] == "Duel") & (df["result_name"] == "Unsuccessful")

    # 상대가 소유권 획득하는 이벤트
    cond_gain = df["event_simple"] == "Duel_Turnover"

    # Dead ball turnover
    cond_deadball = df["event_simple"] == "Error_Out"

    df["is_turnover"] = (
        cond_fail_pass |
        cond_takeon_fail |
        cond_duel_fail |
        cond_gain |
        cond_deadball
    ).astype(int)

    return df

### 8.2.2 episode 내 좌표 차이 / 시간 차이 등 계산 함수

In [ ]:
def add_movement_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    episode 내 start_x, start_y 기준으로 dx, dy, distance, angle, dt 등 추가.
    """
    df = df.copy()
    df = sort_events(df)

    # episode 기준 diff
    df["dx"] = df.groupby("game_episode")["start_x"].diff().fillna(0.0)
    df["dy"] = df.groupby("game_episode")["start_y"].diff().fillna(0.0)
    df["distance"] = np.sqrt(df["dx"]**2 + df["dy"]**2)

    # angle (-pi ~ pi). dx=0,dy=0인 경우는 0으로 처리
    df["angle"] = np.arctan2(df["dy"], df["dx"]).fillna(0.0)

    # 시간 차이
    df["dt"] = df.groupby("game_episode")["time_seconds"].diff().fillna(0.0)

    # episode 내 step index (0~len-1)
    df["step_idx"] = df.groupby("game_episode").cumcount()
    df["epi_len"] = df.groupby("game_episode")["step_idx"].transform("max") + 1
    df["step_idx_norm"] = df["step_idx"] / (df["epi_len"] - 1).replace(0, 1)

    # episode 내 상대 시간 (0~1)
    t_min = df.groupby("game_episode")["time_seconds"].transform("min")
    t_max = df.groupby("game_episode")["time_seconds"].transform("max")
    df["time_rel"] = (df["time_seconds"] - t_min) / (t_max - t_min).replace(0, 1)

    return df

### 8.2.3 zone / 골 방향 feature 함수

In [ ]:
def add_categorical_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 단순화
    df["event_simple"] = df["type_name"].apply(simplify_event)
    df["result_simple"] = df["result_name"].apply(simplify_result)

    # zone
    df["zone_x"] = df["start_x"].apply(get_zone_x)
    df["zone_y"] = df["start_y"].apply(get_zone_y)

    # 골대 기준 거리/각도 (오른쪽 골대 기준)
    goal_x, goal_y = 105.0, 34.0
    df["dist_to_goal"] = np.sqrt((goal_x - df["start_x"])**2 +
                                 (goal_y - df["start_y"])**2)
    goal_angle = np.arctan2(goal_y - df["start_y"],
                            goal_x - df["start_x"])
    df["angle_to_goal"] = goal_angle

    return df

### 8.2.4 Episode 누적 이동량 계산 함수

In [ ]:
def add_episode_cumulative_movement(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = sort_events(df)

    df["cum_dx"] = df.groupby("game_episode")["dx"].cumsum()
    df["cum_dy"] = df.groupby("game_episode")["dy"].cumsum()
    df["movement_norm"] = np.sqrt(df["cum_dx"]**2 + df["cum_dy"]**2)

    return df

### 8.2.5 최종 적용 함수

In [ ]:
def build_event_level_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    train_df나 test_episode_df에 공통 적용할 Event-level Feature Engineering 파이프라인
    """
    df_fe = df.copy()
    df_fe = sort_events(df_fe)
    df_fe = add_turnover_flag(df_fe)
    df_fe = add_movement_features(df_fe)
    df_fe = add_categorical_features(df_fe)
    df_fe = add_episode_cumulative_movement(df_fe)

    return df_fe

## 8.3 에피소드별 Feature Engineering

### 8.3.1 각도 변화량 요약 함수

In [ ]:
def angle_diff(a1, a2):
    diff = a2 - a1
    diff = (diff + np.pi) % (2 * np.pi) - np.pi
    return diff

def compute_angle_smoothness(df: pd.DataFrame) -> pd.DataFrame:
    """
    episode별 angle 변화량 요약 (std, mean_abs, max 등)
    """
    df = df.copy()
    df = sort_events(df)

    records = []

    for ge, g in df.groupby("game_episode"):
        ang = g["angle"].values
        if len(ang) < 3:
            continue

        diffs = [angle_diff(ang[i], ang[i+1]) for i in range(len(ang) - 1)]

        records.append({
            "game_episode": ge,
            "angle_change_std": np.std(diffs),
            "angle_change_mean_abs": np.mean(np.abs(diffs)),
            "angle_change_max": np.max(np.abs(diffs)),
            "angle_change_N": len(diffs),
        })

    angle_df = pd.DataFrame(records)
    return angle_df

def add_angle_smoothness_to_epi(epi_feat: pd.DataFrame,
                                angle_smooth_df: pd.DataFrame) -> pd.DataFrame:

    res = epi_feat.merge(angle_smooth_df, on="game_episode", how="left")
    # 결측은 0 또는 평균값으로 채워도 됨 (길이가 짧은 에피소드)
    res[["angle_change_std", "angle_change_mean_abs",
         "angle_change_max", "angle_change_N"]] = \
        res[["angle_change_std", "angle_change_mean_abs",
             "angle_change_max", "angle_change_N"]].fillna(0.0)

    return res

### 8.3.2 episode별 요약 함수

In [ ]:
def extract_episode_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    episode별 요약 feature (len, ratio_pass/carry, dx/dy, dist 등)
    """
    df = df.copy()
    df = sort_events(df)

    feats = []

    for ge, g in df.groupby("game_episode"):
        types = g["type_name"].values
        xs = g["start_x"].values
        ys = g["start_y"].values

        dx = np.diff(xs)
        dy = np.diff(ys)
        distance = np.sqrt(dx*dx + dy*dy)
        angle = np.arctan2(dy, dx)

        len_epi = len(g)

        feat = {
            "game_episode": ge,
            "epi_len": len_epi,
            "ratio_pass": np.mean(types == "Pass"),
            "ratio_carry": np.mean(types == "Carry"),
            "ratio_turnover": np.mean(g["is_turnover"].values) if len_epi > 0 else 0,

            "dx_mean": np.mean(dx) if len(dx) > 0 else 0.0,
            "dy_mean": np.mean(dy) if len(dy) > 0 else 0.0,
            "angle_mean": np.mean(angle) if len(angle) > 0 else 0.0,
            "angle_std": np.std(angle) if len(angle) > 0 else 0.0,

            "dist_mean": np.mean(distance) if len(distance) > 0 else 0.0,
            "dist_cum": np.sum(distance) if len(distance) > 0 else 0.0,

            # Episode 시작/끝 zone
            "start_zone_x": get_zone_x(xs[0]),
            "start_zone_y": get_zone_y(ys[0]),
        }

        feats.append(feat)

    epi_feat = pd.DataFrame(feats)
    return epi_feat

### 8.3.3 episode별 event entropy 추가 함수

In [ ]:
def add_episode_entropy(df: pd.DataFrame, epi_feat: pd.DataFrame) -> pd.DataFrame:
    """
    episode별 event_simplified entropy 계산 후 epi_feat에 merge
    """
    df = df.copy()
    df = sort_events(df)

    entropy_records = []
    for ge, g in df.groupby("game_episode"):
        seq = g["event_simple"].tolist()
        ent = sequence_entropy(seq)
        entropy_records.append({"game_episode": ge, "entropy_event": ent})

    ent_df = pd.DataFrame(entropy_records)

    epi_feat = epi_feat.merge(ent_df, on="game_episode", how="left")
    return epi_feat

### 8.3.4 Episode summary 통합 최종 함수

In [ ]:
def build_episode_level_features(df_fe: pd.DataFrame) -> pd.DataFrame:
    """
    Event별 FE가 적용된 df_fe를 입력 받아,
    episode별 summary feature를 생성.
    """
    epi_feat = extract_episode_summary(df_fe)
    angle_smooth_df = compute_angle_smoothness(df_fe)
    epi_feat = add_angle_smoothness_to_epi(epi_feat, angle_smooth_df)
    epi_feat = add_episode_entropy(df_fe, epi_feat)

    return epi_feat